In [117]:
import pandas as pd
import numpy as np
import re
import torch
from torch.nn import CrossEntropyLoss
from core import get_utterance_by_TVID, askChatGPT, num_tokens_from_messages

# Train

In [118]:
msg_train_data = pd.read_csv("../data/CPED/train_split.csv")
msg_train_data = msg_train_data.iloc[0:50]
# msg_train_data = msg_train_data[msg_train_data["TV_ID"] == 23]
msg_train_data.head(2)

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,1,01_000,01_000_000,童文洁,female,middle-aged,high,high,low,low,high,other-venue,108_136,156_202,neutral,neutral,greeting,真巧
1,1,01_000,01_000_001,童文洁,female,middle-aged,high,high,low,low,high,other-venue,193_144,253_197,neutral,neutral,greeting,车没事了


In [119]:

res = [{
    "role": "system",
    "content": "接下来，我将提供给你一段对话，这段对话是一个人说的,不过场景和说话对象都不相同。格式如下。'\n童文洁: 真巧\n童文洁: 车没事了\n童文洁: 没事没事 你也去春风\n'。然后你需要提供对话人物的大五人格的维度属于高还是低。比如你需要回复，'童文洁: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 low\n刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'"
}] # 结果保存
for i in get_utterance_by_TVID(msg_train_data):
    res.append({
        "role": "user",
        "content": i[0]
    })
    if not len(i[1]) == 0:
        res.append({
            "role": "assistant",
            "content": i[1]
        })
num_tokens_from_messages(res)

1195

# Test

In [120]:
msg_test_data = pd.read_csv("../data/CPED/test_split.csv")
# msg_test_data = msg_test_data[msg_test_data["TV_ID"] == 27]
msg_test_data.head(2)

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Scene,FacePosition_LU,FacePosition_RD,Sentiment,Emotion,DA,Utterance
0,27,27_000,27_000_000,林宛瑜,female,young,low,high,high,high,high,car,488_14,654_204,neutral,neutral,statement-non-opinion,好了没事了
1,27,27_000,27_000_001,林宛瑜,female,young,low,high,high,high,high,car,481_47,613_244,neutral,neutral,thanking,谢谢你哦


In [ ]:
y = [] # 真实结果和预测结果
asd = []
for i in get_utterance_by_TVID(msg_test_data):
    mm = [v for v in res]
    mm.append({
        "role": "user",
        "content": i[0]
    })
    asd.append(mm)
    if num_tokens_from_messages(mm) < 8192 and num_tokens_from_messages(mm) > 4000:
        arr = askChatGPT(mm)
        y.append([i[1], arr])

In [129]:
num = 0
ress = []
for i,v in enumerate(asd):
    c = num_tokens_from_messages(v)
    print(i, c)
    if c < 8192 and c > 4000:
        num += 1
        ress.append(i)

print("最后分析人数为", num)
print(ress)

0 5764
1 28004
2 55486
3 4509
4 22196
5 38799
6 2416
7 50137
8 6843
9 21109
10 21723
11 8266
12 7797
13 15940
14 3321
15 39464
16 13301
17 5875
18 9417
19 1620
20 3575
21 40460
22 4478
23 8305
24 2171
25 2805
26 13695
27 13442
28 1900
29 17722
30 18852
31 5759
32 38886
33 9058
最后分析人数为 7
[0, 3, 8, 12, 17, 22, 31]


In [131]:
asd[22]

[{'role': 'system',
  'content': "接下来，我将提供给你一段对话，这段对话是一个人说的,不过场景和说话对象都不相同。格式如下。'\n童文洁: 真巧\n童文洁: 车没事了\n童文洁: 没事没事 你也去春风\n'。然后你需要提供对话人物的大五人格的维度属于高还是低。比如你需要回复，'童文洁: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 low\n刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'"},
 {'role': 'user',
  'content': '刘静: 是你呀 没事了没事 谢谢你\n刘静: 对\n刘静: 对呀\n刘静: 高三了\n刘静: 三班啊\n刘静: 我儿子叫季杨杨\n刘静: 那是他舅舅\n'},
 {'role': 'assistant',
  'content': '刘静: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high\n'},
 {'role': 'user',
  'content': '方一凡: 妈\n方一凡: 妈，我错了\n方一凡: 数学92\n方一凡: 化学91\n方一凡: 除了英语一般般\n方一凡: 其他都特别棒\n方一凡: 我随我爸爸\n'},
 {'role': 'assistant',
  'content': '方一凡: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low\n'},
 {'role': 'user',
  'content': '童文洁: 真巧\n童文洁: 车没事了\n童文洁: 没事没事 你也去春风\n童文洁: 你孩子也在这上学\n童文洁: 真巧 我儿子也在这\n童文洁: 你孩子几年级\n童文洁: 哪个班\n童文洁: 我儿子也三班\n童文洁: 我儿子叫方一凡\n童文洁: 你孩子叫什么\n童文洁: 季杨杨，好像我听凡凡说过\n童文洁: 对\n童文洁: 是不是每次开家长会来的都是他爸爸\n童文洁: 我说呢这么年轻\n童文洁: 你不要叫我妈\n童文洁: 我不是你妈\n童文洁: 你没错\n童文洁: 我错了\n童文洁: 我为什么要生你\n童文洁: 我吃饱了撑的我就不该生你\n童文洁: 方一凡我真的

In [125]:
y_true = pd.DataFrame({
    "name": [],
    "Neuroticism": [],
    "Extraversion": [],
    "Openness": [],
    "Agreeableness": [],
    "Conscientiousness": []
})
y_pre = pd.DataFrame({
    "name": [],
    "Neuroticism": [],
    "Extraversion": [],
    "Openness": [],
    "Agreeableness": [],
    "Conscientiousness": []
})
for i in y:
    i1 = re.findall(
        r"(.*?)[:]\s神经质\s(.*?),\s外倾性\s(.*?),\s开放性\s(.*?),\s亲和性\s(.*?),\s尽责性\s(.*?)$", i[0])[0]
    y_true = pd.concat([y_true, pd.DataFrame({
        "name": [i1[0]],
        "Neuroticism": [1 if i1[1] == "high" else 0],
        "Extraversion": [1 if i1[2] == "high" else 0],
        "Openness": [1 if i1[3] == "high" else 0],
        "Agreeableness": [1 if i1[4] == "high" else 0],
        "Conscientiousness": [1 if i1[5] == "high" else 0]
    })], axis=0)
    i2 = re.findall(
        r"(.*?)[:]\s神经质\s(.*?),\s外倾性\s(.*?),\s开放性\s(.*?),\s亲和性\s(.*?),\s尽责性\s(.*?)$", i[1])[0]
    y_pre = pd.concat([y_pre, pd.DataFrame({
        "name": [i2[0]],
        "Neuroticism": [1 if i2[1] == "high" else 0],
        "Extraversion": [1 if i2[2] == "high" else 0],
        "Openness": [1 if i2[3] == "high" else 0],
        "Agreeableness": [1 if i2[4] == "high" else 0],
        "Conscientiousness": [1 if i2[5] == "high" else 0]
    })], axis=0)

In [126]:
y_pre_torch = torch.from_numpy(y_pre.iloc[:, 1:].to_numpy())
y_true_torch = torch.from_numpy(y_true.iloc[:, 1:].to_numpy())

cce = CrossEntropyLoss()
print("多分类交叉熵 值为", cce(y_pre_torch, y_true_torch))

多分类交叉熵 值为 tensor(5.0709, dtype=torch.float64)


### 真实
吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low
曾小贤: 神经质 high, 外倾性 high, 开放性 low, 亲和性 high, 尽责性 low
林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
胡一菲: 神经质 low, 外倾性 high, 开放性 low, 亲和性 low, 尽责性 high
陆展博: 神经质 high, 外倾性 low, 开放性 high, 亲和性 high, 尽责性 high
其他: 神经质 unknown, 外倾性 unknown, 开放性 unknown, 亲和性 unknown, 尽责性 unknown

### GPT
吕子乔: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 low
曾小贤: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
林宛瑜: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
胡一菲: 神经质 high, 外倾性 high, 开放性 high, 亲和性 low, 尽责性 high
陆展博: 神经质 low, 外倾性 high, 开放性 high, 亲和性 high, 尽责性 high
其他: 神经质 low, 外倾性 low, 开放性 low, 亲和性 low, 尽责性 low